## Image Analysis

In summer, beaches are monitored by lifeguards, who gather information about the occupancy, sea
state, wind, etc. Image processing techniques can be used to automate this task. In this project, we will
estimate the amount of people present in a beach (in computer vision, “crowd counting”), focusing on
all aspects of an image processing project: data annotation, algorithm design and implementation, and
validation.

### Import useful libraries and files

In [ ]:
import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imutils
import os
from sklearn.metrics import mean_squared_error, precision_score



path = "/Users/amarachianyim/Documents/EDISS/UIB/1st_SEM/Image_and _Video_Analysis/Anyim/"
results = path+'results/'
#read annotations and select the useful columns (label, (x,y) cordinates of the points, and filename)
df = pd.read_csv(path+'labels_image_analysis.csv', header=None, usecols=[0,1,2,3], names=['label', 'x', 'y', 'filename'])
#summarizes the number of people in each image
ground_truth = df.groupby('filename').size().reset_index(name='people')

df.head(10), ground_truth

### Define functions to be used in the algorithm

In [ ]:
def get_region_of_interest(image):
    height, width = image.shape[:2]
    ROI= np.array([[(0,height),(0,450),(1920,450),(1920,height)]], dtype= np.int32)
    blank= np.zeros_like(image)
    region_of_interest= cv2.fillPoly(blank, ROI,255)
    region_of_interest_image= cv2.bitwise_and(image, region_of_interest)
    return region_of_interest_image

def detect_edges(image):
    edges = cv2.Canny(image,100,200,3)
    return edges

def binarize(image):
    th, im_th = cv2.threshold(image, 50, 250, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return im_th

def gaussian_blur(gray):
    blur = cv2.GaussianBlur(gray, (7,7), sigmaX=33, sigmaY=33)
    return blur

def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    dilation = cv2.dilate(image,kernel,iterations = 3)
    return dilation

#check if point is in a contour
def point_in_rect(point,rect):
    x1, y1, w, h = rect
    x2, y2 = x1+w, y1+h
    x, y = point
    if (x1 < x and x < x2):
        if (y1 < y and y < y2):
            return True
    return False


def get_contour(image, im, n, results):
    contours, hierarchy  = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img = cv2.drawContours(im, contours, -1, (0,255,75), 2)
    contoure = 0

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        #this is to get rid of the countour that it bounding the region of interest
        if w<1920:
            im= cv2.rectangle(im, (x, y), (x + w, y + h), (0, 0, 255), 2)
            for i in coord:
                if point_in_rect(i, (x, y, w, h)):
                    contoure +=1

    #get the true positives if the countour actually has a point in it
    true_pos.append(contoure)
    # 1 is subtracted beacuse of the contour that spans the entire width of the image
    number_of_people_in_image= len(contours)-1
    #getting the estimated count
    y_pred.append(number_of_people_in_image)
    #Please uncomment the line below to visualize the contours in each image
#     cv2.imshow('contour'+str(n),im)

#To save the files, uncomment the line below
    #cv2.imwrite(results+str(n), im)


### Algorithm

In [ ]:
#Initialize
y_pred = []
y_true = []
true_pos = []
for i in ground_truth['people']:
    y_true.append(i)
    
for i in ground_truth['filename']:
    coord = []
    df2=df.loc[df['filename'] == i]
    for index, row in df2.iterrows():
        coord.append((row['x'], row['y']))
    im = cv2.imread(path+i)
    #defining the region of interest
    ROI = get_region_of_interest(im)
    #grayscale the image
    gray = cv2.cvtColor(ROI, cv2.COLOR_BGR2GRAY)
    #smoothening
    blur = gaussian_blur(gray)
    #binarization
    im_th = binarize(blur)
    #edge detection
    edges = detect_edges(im_th)
    #Dilation
    dilated = dilate(edges)
    #Count people in the image
    get_contour(dilated, im, i, results)

#Please uncomment the lines below to visualize the contours in each image
# cv2.waitKey(0) 
# cv2.destroyAllWindows()




### Validation

In [ ]:
y_pred, true_pos, y_true

In [ ]:
mean_squared_error(y_true, y_pred)

In [ ]:
#Visualize the actual vs predicted number of people
fig, ax = plt.subplots()

ax.plot(y_true, color = 'green', label = 'Actual')
ax.plot(y_pred, color = 'red', label = 'Predicted')
ax.legend(loc = 'upper left')
ax.set_xlabel('image index')
ax.set_ylabel('count')
plt.show()